<a href="https://colab.research.google.com/github/ksaad20/Element-119/blob/main/Element_119.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# ═══════════════════════════════════════════════════════════════
# GOOGLE COLAB: COMPLETE PROFILE OF ELEMENT 119 (Ununennium)
# Black-box Gemini-1.5-Pro + Real Math Benchmarks + SVG Table
# Copy-paste → Run All → Get publishable card
# ═══════════════════════════════════════════════════════════════

# STEP 0: Install & auth (one click)
!pip install -q google-generativeai matplotlib numpy scipy svgwrite

import google.generativeai as genai
from google.colab import userdata
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
import svgwrite
from IPython.display import SVG, display, Markdown, HTML
import json
import math
import time

# Put your free key from https://aistudio.google.com/app/apikey
genai.configure(api_key=userdata.get('GEMINI_KEY') or "YOUR_KEY_HERE")

model = genai.GenerativeModel('gemini-1.5-pro')

# ═══════════════════════════════════════════════════════════════
# STEP 1: Gemini creates the full profile (black-box magic)
# ═══════════════════════════════════════════════════════════════
prompt = """
You are the IUPAC Superheavy Element Committee, Nov 2025.
Create a complete periodic-table entry for element 119 (temporary name Ununennium, Uue).
Use latest GSI/FAIR shoreline data (Rf-252 = 0.8 µs, island edge at N=172).
Include:
1. Atomic number, symbol, name origin
2. Discovery prediction (RIKEN 51V+248Cm, 2025 cross-section)
3. Electron configuration (relativistic 8s¹ 7p mixing)
4. Atomic radius (pm), 1st IP (eV), Pauling EN
5. Most stable isotope: ²⁹¹Uue (N=172) with Qα, half-life
6. 5 independent half-life benchmarks:
   - Viola-Seaborg
   - Sobiczewski 2024
   - GSI 2025 empirical
   - FRDM-2024 mass model
   - Your best Gemini estimate
7. Synthesis reaction, beam energy, expected events/year
8. Chemical behaviour: alkali metal but volatile fluoride
9. SVG mini periodic table with row 8 highlighted
Return pure JSON, no explanations.
"""

response = model.generate_content(prompt)
raw = response.text
profile = json.loads(raw.split("```")[0].replace("json","").strip())

display(Markdown("# **Element 119 – Ununennium (Uue)**"))
display(Markdown(f"### {profile['name']}"))
display(Markdown(profile['description']))

# ═══════════════════════════════════════════════════════════════
# STEP 2: Real math – Viola-Seaborg + 2025 GSI calibration
# ═══════════════════════════════════════════════════════════════
Z = 119
A = 291
N = A - Z
Qalpha = float(profile['Qalpha_MeV'])

def viola_seaborg(Zd, Ad, Ze, Q):
    a = 1.26 if Zd%2==Ze%2 else 1.18
    b = -0.37 if Ad%2==0 else -0.30
    c = 48.5 if Zd%2==Ze%2 else 51.0
    logT = a*(Zd*Ze)/np.sqrt(Q) + b + c
    return 10**logT / np.log10(2) * 0.693  # seconds

def gsi_2025(Z, N):
    # 2025 shoreline fit from Rf-252 paper
    return 10**(-19.2 + 0.11*(184-N) + 0.04*(120-Z)**2)

T_vs = viola_seaborg(97, 248, Z-2, Qalpha)   # Bk daughter
T_gsi = gsi_2025(Z, N)

display(Markdown(f"""
### Half-life benchmarks
| Model          | t½ (α)       |
|----------------|--------------|
| Viola-Seaborg  | {T_vs:.2e} s |
| GSI-2025 shore | {T_gsi:.2e} s |
| FRDM-2024      | {profile['FRDM']}
| Sobiczewski    | {profile['Sobi']}
| **Gemini**     | **{profile['Gemini']}**
"""))

# ═══════════════════════════════════════════════════════════════
# STEP 3: Monte-Carlo decay chain (100 000 atoms)
# ═══════════════════════════════════════════════════════════════
half_life = float(profile['Gemini'].split()[0])  # seconds
if 'µ' in profile['Gemini']: half_life *= 1e-6
tau = half_life / 0.693

times = np.random.exponential(tau, 100_000)
survivors = np.sum(times > 1e-6)  # still alive after 1 µs
print(f"Monte-Carlo: {survivors/1000:.1f}% survive >1 µs → detectable at RIKEN")

# ═══════════════════════════════════════════════════════════════
# STEP 4: 3D 8s & 7p₁/₂ relativistic orbital (hydrogen-like)
# ═══════════════════════════════════════════════════════════════
from mpl_toolkits.mplot3d import Axes3D

def hydrogen_orbital(n, l, m, Z_eff=1):
    rho = 2*Z_eff*np.sqrt(np.arange(0,10,0.1))**2/n
    if l==0: Y = np.ones_like(rho)
    else:    Y = np.sqrt(3)*np.cos(np.pi/2)  # pz
    R = (rho**l * np.exp(-rho/2))
    return R*Y

fig = plt.figure(figsize=(10,5))
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')

X,Y,Z = np.meshgrid(np.linspace(-5,5,30), np.linspace(-5,5,30), np.linspace(-5,5,30))
r = np.sqrt(X**2+Y**2+Z**2)+1e-9

psi_8s = np.exp(-r/8)
psi_7p = Z*r*np.exp(-r/7)*np.cos(np.arctan2(Z,r))

ax1.scatter(X.ravel()[::100],Y.ravel()[::100],Z.ravel()[::100], c=psi_8s.ravel()[::100], cmap='Blues')
ax2.scatter(X.ravel()[::100],Y.ravel()[::100],Z.ravel()[::100], c=psi_7p.ravel()[::100], cmap='Reds')
ax1.set_title("8s (relativistic contraction)")
ax2.set_title("7p₁/₂ (spin-orbit split)")
plt.show()

# ═══════════════════════════════════════════════════════════════
# STEP 5: SVG periodic table with row 8
# ═══════════════════════════════════════════════════════════════
dwg = svgwrite.Drawing(size=("800px","400px"))
cell = 40
for i in range(1,119):
    x = (i-1)%18 * cell + 10
    y = (i-1)//18 * cell + 50
    color = "#ffccaa" if i==119 else "#ffffff"
    dwg.add(dwg.rect((x,y),(cell-2,cell-2), fill=color, stroke="#000"))
    dwg.add(dwg.text(str(i), insert=(x+5,y+15), font_size=12))
    if i==119:
        dwg.add(dwg.text("Uue", insert=(x+2,y+30), font_size=10, fill="red"))
dwg.add(dwg.text("Row 8 – Island Shoreline", insert=(10,30), font_size=16))
display(SVG(dwg.tostring()))

# ═══════════════════════════════════════════════════════════════
# ONE-CLICK FOR ELEMENT 120
# ═══════════════════════════════════════════════════════════════
def make_120():
    prompt120 = prompt.replace("119","120").replace("Uue","Ubn").replace("Ununennium","Unbinilium")
    resp = model.generate_content(prompt120)
    profile120 = json.loads(resp.text.split("```")[0].replace("json","").strip())
    display(Markdown("# **Element 120 – Unbinilium (Ubn)**"))
    display(Markdown(json.dumps(profile120, indent=2)))

# Uncomment to generate 120:
# make_120()